Vamos usar o pacote pnadc com documentação disponivel em https://pnadc.readthedocs.io/en/latest/ . Esse pacote permite acessar diretamente os dados da PNAD continua, ou pnadc. A partir da extração e filtro vamos analisar os dados do ultimo trimestre de 2024, coeltados de 01/10/2024 a 31/12/2024.

Estou em duvida quanto a V403331 Tipo de rendimento/retirada  em benefícios que recebia/fazia nesse trabalho
1	Pessoa recebendo somente em beneficios, exceto aprendizado
2	Aprendiz ou estagiário recebendo em aprendizado e outros beneficios
3	Aprendiz ou estagiário recebendo somente em aprendizado
    Não aplicável

Vou testar como está mas fica a ideia de acrescentar

In [9]:
#pip install pynad
#pip install pnadc

import pnadc
import pandas as pd
import numpy as np

In [4]:

import pnadc
# Variaveis
pesos_replicacao = ['V1028' + str(i).zfill(3) for i in range(1,201)]

# Variáveis de interesse + pesos
variaveis = [
    "Ano",
    "Trimestre",
    "UF",
    "V1028",    # Peso
    "V2009",   # Idade
    "V3003A",   # Escolaridade
    "VD4007",   # Posição de ocupação #1 empregado
    "VD4009",    # Posição na ocupação
    "VD4013"    # Horas trabalhadas
    #"V403331"   # Tipo de rendimento/beneficios
]  + pesos_replicacao                            


In [ ]:
# Definir caminho para o arquivo otimizado
caminho_arquivo = 'pnad_filtrada.parquet'

# Verificar se o arquivo otimizado já existe
if os.path.exists(caminho_arquivo):
    print("🔄 Lendo dados do arquivo otimizado...")
    dados = pd.read_parquet(caminho_arquivo)
else:
    print("⬇️ Baixando dados da PNAD e salvando em formato otimizado...")
    # Importar dados online
    dados = pnadc.get(quarter=4, year=2024, keep_columns=variaveis)

    # Salvar em formato otimizado (Parquet)
    dados.to_parquet(caminho_arquivo)
    print("✅ Dados salvos como 'pnad_filtrada.parquet'")


In [5]:
# Importar dados online
dados = pnadc.get(quarter=4, year=2024, keep_columns=variaveis)

caminho_arquivo = 'pnad_filtrada.parquet'
dados.to_parquet(caminho_arquivo)

Chaves_PNADC.pdf download is complete!
Definicao_variaveis_derivadas_PNADC_20200211.pdf download is complete!
Deflatores.zip download is complete!
Unziping Deflatores.zip
Unzip complete
Dicionario_e_input_20221031.zip download is complete!
Unziping Dicionario_e_input_20221031.zip
Unzip complete
Estrutura_Atividade_CNAE_Domiciliar_2_0.xls download is complete!
Estrutura_Ocupacao_COD.xls download is complete!
Grupamentos_ocupacionais_atividades_PNADC.pdf download is complete!
PNADcIBGE_Deflator_Trimestral.pdf download is complete!
Variaveis_PNADC_Trimestral.xls download is complete!
PNADC_042024.zip download is complete!
Unziping PNADC_042024.zip
Unzip complete
Building PNADC_042024.txt
Multiprocessing chunks


100%|██████████| 24/24 [02:52<00:00,  7.17s/it]


Done!


TENTAR COM BD

In [ ]:
# Filtrar faixa etária de 16 a 25 anos
dados_filtrados = dados[(dados['V2009'] >= 16) & (dados['V2009'] <= 25)]

# Filtrar escolaridade (Ensino Médio completo ou Superior incompleto)
escolaridade_codes = [ 6,7,8] # Ok!
dados_filtrados = dados_filtrados[dados_filtrados['V3003A'].isin(escolaridade_codes)]

# Filtrar posição na ocupação (Empregado)
ocupacao_codes = [1]
dados_filtrados = dados_filtrados[dados_filtrados['VD4007'].isin(ocupacao_codes)]

# Filtrar tipo de ocupação  (Empregado setor publico ou privado)
tipo_ocupacao_codes = [1,3]
dados_filtrados = dados_filtrados[dados_filtrados['VD4009'].isin(tipo_ocupacao_codes)]

# Filtrar horas trabalhadas habitualmente   (Até 39 horas)
hora_codes = [1,2]
dados_filtrados = dados_filtrados[dados_filtrados['VD4013'].isin(hora_codes)]

# Estimativa pontual usando peso básico
total_estagiarios = dados_filtrados['V1028'].sum()
print(f"Estimativa do número de estagiários: {total_estagiarios:,.0f}")

# Estimativas com pesos de replicação
replicates = [dados_filtrados[peso].sum() for peso in pesos_replicacao]

# Calcular variância seguindo metodologia do IBGE (Fay’s method com k=0.5 geralmente)
variancia = np.mean([(rep - total_estagiarios)**2 for rep in replicates]) * 4

# Erro padrão e intervalo de confiança
erro_padrao = np.sqrt(variancia)
ic_inferior = total_estagiarios - 1.96 * erro_padrao
ic_superior = total_estagiarios + 1.96 * erro_padrao

# Exibir resultados
print(f"Estimativa do número de estagiários: {total_estagiarios:,.0f}")
print(f"Erro padrão: {erro_padrao:,.0f}")
print(f"Intervalo de Confiança (95%): [{ic_inferior:,.0f}, {ic_superior:,.0f}]")

Estimativa do número de estagiários: 405,581
Estimativa do número de estagiários: 405,581
Erro padrão: 36,601
Intervalo de Confiança (95%): [333,843, 477,318]


Para PNAD cont vamos usar Samplics com estimação por replicação, usando metodo de bootstrap que e o da pnad continua o parametro nosso vai ser ? nao sei preciso pensar, ele calcula media da renda, eu quero contagem


CONFERIR CODIGOS NO DICIONARIO PNADC